In [1]:
%%capture
# ignore warnings in this cell
%load_ext autoreload
%autoreload 2

import os
os.environ["USE_NUMBA"] = "1"
import random

from eals.eals import ElementwiseAlternatingLeastSquares, load_model
from eals.util import create_user_items

In [2]:
# Data preparation
user_count = 2000
item_count = 1000
data_count = user_count * 20
new_user_count = 200
new_item_count = 100

user_items = lambda: create_user_items(
    user_count=user_count + new_user_count,
    item_count=item_count + new_item_count,
    data_count=data_count,
)

In [3]:
# Benchmarking fit()

model = ElementwiseAlternatingLeastSquares(max_iter=10)
model.fit(user_items())  # warm up
%time model.fit(user_items())

CPU times: user 997 ms, sys: 96.4 ms, total: 1.09 s
Wall time: 172 ms


In [4]:
# Benchmarking update_user()

# existing users only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count))  # warm up
%timeit model.update_user(random.randrange(user_count))

# new users only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count, user_count + new_user_count))  # warm up
%timeit model.update_user(random.randrange(user_count, user_count + new_user_count))

# mixed users
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count + new_user_count))  # warm up
%timeit model.update_user(random.randrange(user_count + new_user_count))

26.3 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.18 µs ± 86.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
25.4 µs ± 736 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
# Benchmarking update_item()

# existing items only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count))  # warm up
%timeit model.update_item(random.randrange(item_count))

# new items only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count, item_count + new_item_count))  # warm up
%timeit model.update_item(random.randrange(item_count, item_count + new_item_count))

# mixed items
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count + new_item_count))  # warm up
%timeit model.update_item(random.randrange(item_count + new_item_count))

33.7 µs ± 208 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.24 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
31.8 µs ± 390 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# Benchmarking update_model()
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_model(random.randrange(user_count), random.randrange(item_count))  # warm up
%timeit -n 1000 model.update_model(random.randrange(user_count), random.randrange(item_count))

130 µs ± 7.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
# Benchmarking calc_loss() for csr matrices
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.calc_loss()  # warm up
%timeit model.calc_loss()

3.38 ms ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# Benchmarking calc_loss() for lil matrices
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model._convert_data_for_online_training()
model.calc_loss()  # warm up
%timeit model.calc_loss()

13.5 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%run benchmark.py -f 2020-08-24-view0100.json -m csr

Benchmarking calc_loss()
real data (2020-08-24-view0100.json), csr matrix
  load data: (use cache: 2020-08-24-view0100-train_data.npy) 2.502926826477051 sec
    user_count=1200777, item_count=110547, nnz=37342551
  setup: 3.760756015777588 sec
  elapsed: 4.434885263442993 sec


In [10]:
%run benchmark.py -f 2020-08-24-view0100.json -m lil

Benchmarking calc_loss()
real data (2020-08-24-view0100.json), lil matrix
  load data: (use cache: 2020-08-24-view0100-train_data.npy) 2.098931074142456 sec
    user_count=1200777, item_count=110547, nnz=37342551
  setup: 30.239140033721924 sec
  elapsed: 11.197751760482788 sec


In [11]:
# Model serializetion benchmark
# 2020-08-24-view0100.json と大体同じサイズのランダムデータ
ui = create_user_items(
    user_count=1200000,
    item_count=110000,
    data_count=37000000
)
model = ElementwiseAlternatingLeastSquares()
model.init_data(ui)
model._convert_data_for_online_training()

print("Serialization without compression")
%time model.save("model.json", compress=False)
print("Deserialization without compression")
%time model = load_model("model.json")
print("Serialization with compression")
%time model.save("model.json.gz", compress=True)
print("Deserialization with compression")
%time model = load_model("model.json.gz")

Serialization without compression
CPU times: user 6.54 s, sys: 2.76 s, total: 9.29 s
Wall time: 9.67 s
Deserialization without compression
converting type of user_items to np.float32
CPU times: user 1min 36s, sys: 1min 53s, total: 3min 30s
Wall time: 4min 15s
Serialization with compression
CPU times: user 6min 57s, sys: 4.95 s, total: 7min 2s
Wall time: 7min 7s
Deserialization with compression
converting type of user_items to np.float32
CPU times: user 1min 50s, sys: 2min 9s, total: 4min
Wall time: 4min 41s
